In [1]:
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import pygame
from gtts import gTTS
from time import sleep
import speech_recognition as sr
import tempfile
import datetime

# Suppress FutureWarning for tokenization spaces
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")



c:\Users\TEMP\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.6.1 (SDL 2.28.4, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Load the DistilGPT model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        tts.save(temp_audio_file.name)
        temp_file_path = temp_audio_file.name
    pygame.mixer.init()
    try:
        pygame.mixer.music.load(temp_file_path)
        pygame.mixer.music.play()

        # Check music play status at intervals
        while pygame.mixer.music.get_busy():
            pygame.time.wait(50)

    except Exception as e:
        print(f"Error playing audio: {e}")
    finally:
        pygame.mixer.quit()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)


In [ ]:
def capture_speech(mic_index):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        print("Listening for your query...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=6)
        except sr.WaitTimeoutError:
            print("Listening timed out. Please speak again.")
            return None

    try:
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with Google Speech Recognition service; {e}")
        return None


In [ ]:
def generate_response(user_input):
    if "what is your name" in user_input.lower():
        return "I am Amigo, your all-weather conversational companion, just like the bike that you are riding on."
    
    # Check for factual or current event queries
    if any(keyword in user_input.lower() for keyword in ["what is", "who is", "when is", "where is", "current", "today", "news"]):
        return "Thank you for your query, but that's out of my scope of training."

    # Encode user input
    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=50)
    
    # Generate response with parameters tuned to reduce repetition
    reply_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        num_beams=5,  # Beam search for better context handling
        early_stopping=True,  # Stops generation when end of meaningful output is reached
        no_repeat_ngram_size=2  # Prevents repetition of phrases
    )
    
    # Decode the response
    bot_response = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    
    return bot_response


In [ ]:
def handle_time_date_queries(user_input):
    if "time" in user_input:
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif "date" in user_input or "today" in user_input:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return f"Today is {current_date}."
    else:
        return None


In [ ]:
def select_microphone():
    mic_list = sr.Microphone.list_microphone_names()
    index = 2  # Select microphone index
    return index


In [ ]:
def listen_for_wake_word(mic_index):
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=mic_index)
    
    print("Waiting for wake word 'Hey Amigo'...")
    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source, timeout=5)
            
        try:
            user_input = recognizer.recognize_google(audio)
            if "hey amigo" in user_input.lower():
                print("Wake word detected!")
                return True
        except sr.UnknownValueError:
            continue
        except sr.RequestError as e:
            print(f"Error with Google Speech Recognition service: {e}")
            continue


In [ ]:
def amigo_conversational_companion():
    mic_index = select_microphone()

    # Step 1: Wait for the wake word
    if listen_for_wake_word(mic_index):
        # Step 2: Greet the user
        greeting = "Hi, I am Amigo. How can I help you today?"
        print(f"Amigo: {greeting}")
        speak_response(greeting)
        
        # Step 3: Enter conversation loop
        while True:
            user_input = capture_speech(mic_index)
            if user_input is None:
                continue

            # Check if the user wants to exit
            if "exit" in user_input.lower() or "goodbye" in user_input.lower():
                farewell_message = "Goodbye, have a nice day!"
                print(f"Amigo: {farewell_message}")
                speak_response(farewell_message)
                break

            # Handle time/date queries or generate general responses
            bot_response = handle_time_date_queries(user_input) or generate_response(user_input)
            print(f"Amigo: {bot_response}")
            speak_response(bot_response)



In [ ]:
if __name__ == "__main__":
    amigo_conversational_companion()


Waiting for wake word 'Hey Amigo'...
Wake word detected!
Amigo: Hi, I am Amigo. How can I help you today?
Listening for your query...
User said: what is the time now
Amigo: The current time is 06:18 AM.
Listening for your query...
User said: what is today's date
Amigo: Today is Tuesday, October 08, 2024.
Listening for your query...
User said: what is your name and who are you
Amigo: I am Amigo, your all-weather conversational companion, just like the bike that you are riding on.
Listening for your query...
User said: what is the capital of United States of America
Amigo: Thank you for your query, but that's out of my scope of training.
Listening for your query...
User said: give me information about Japan


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Amigo: give me information about Japan.”

“I don't know what I'm talking about, but I can tell you that I've been in Japan for a long time, and I think it's a great place to be. I
Listening for your query...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User said: tell me a joke
Amigo: tell me a joke.”
Listening for your query...
